<script type="text/x-mathjax-config">
MathJax.Hub.Config({
  TeX: { equationNumbers: { autoNumber: "AMS" } }
});
</script>

<h1><center>Machine Learning in Healthcare Informatics</center></h1>

## Temporal Needleman-Wunsch

Consider two temporal sequences:

>$X = t_{x_1}.x_1, t_{x_2}.x_2,..., t_{x_m}x_m$  
<br><br>
$Y = t_{y_1}.y_1, t_{y_2}.y_2,...,t_{y_n}.y_n$, 
<br><br>
where $t_{x_i}$ and $t_{y_j}$ are the transition times for elements $x_i$ and $y_j$, respectively, for $i\in [1,m]$ and $j\in [1,n]$.

The Temporal Needleman-Wunsch, similarly to the Needleman-Wunsch consists on three steps:
1. Initialisation of the score matrix $H$, traceback matrix $T$ and auxiliary matrices $TR$ and $TC$
>\begin{equation}
H_{r0} = -rg; \ \ \ H_{0c} = -cg \quad \forall r \in [0,m], c \in [0,n]
\label{eq:init}
\end{equation}

2. Calculation of scores, filling the traceback matrix and auxiliary matrices
>\begin{equation}
	H_{i,j} = \max 
	\left\{\!\begin{aligned}
&H_{i-1,j-1} + S(x_i,y_j) - f(t_{x_i}+TR_{i-1,j-1},t_{y_j}+TC_{i-1,j-1}) (*)\\[1ex]
&H_{i-1,j} - g \\[1ex]
&H_{i,j-1} - g
\end{aligned}\right\}, \quad \forall i \in [1,m], j \in [1,n]
\label{eq:tnw}
\end{equation}
<br><br>
\begin{equation}
TR_{i,j}=
  \begin{cases}
    0, & \text{if $T_{i,j} =$ 'diag'}\\
    TR_{i,j-1} & \text{if $T_{i,j}=$ 'left'}\\
    TR_{i-1,j} + t_{xi} & \text{if $T_{i,j}=$ 'up'}
  \end{cases}
  \label{eq:TR}
\end{equation}
<br><br>
\begin{equation}
TC_{i,j}=
  \begin{cases}
    0, & \text{if $T_{i,j} =$ 'diag'}\\
    TC_{i,j-1} + t_{yj} & \text{if $T_{i,j}=$ 'left'}\\
    Tc_{i-1,j}  & \text{if $T_{i,j}=$ 'up'}
  \end{cases}
  \label{eq:TC}
\end{equation}
<br><br>
\begin{equation}
T_{i_,j}=
   \begin{cases}
     \text{'diag'}, & \text{if $H_{i,j} =$ *}\\
     \text{'up'} & \text{if $H_{i,j}= $ $H_{i-1,j} - g$}\\
    \text{'left'}  & \text{if $H_{i,j}=$ $H_{i,j-1} - g$}
  \end{cases}
  \label{eq:T}
\end{equation}

3. Deducing the alignment from the traceback matrix

#### User defined parameters:
* $g$: gap penalty
* $ S(x_i,y_j)$ : pre-defined scoring schema
* f : temporal penalty function
> \begin{equation}
    f(t_{x},t_{y}) = T_p\frac{|t_{x}-t_{y}|}{\max(t_{x},t_{y})}
  \end{equation}
<br><br>
where $T_p$ is a constant

### Example

>$ X = 0.A,t_1.B,t_2.C,t_3.D $
<br><br>
$ Y = 0.A,t_4.D $
<br><br>
where $t_4 = t_1+t_2+t_3$
<br><br>
Lets assume $t_1=1,t_2=2,t_3=1$ and  $t_4=4$

For this example the user-defined parameters are:

* $g = 0.5$
* simple scoring system: exact matching events get a score of 1 and events that do not match get a score of -1.1
* $T_p = 0.25$


#### 1. Initialisation

We start building our matrix $H$ using equation \ref{eq:init}. The remaining matrices are computed using equations \ref{eq:TR}, \ref{eq:TC} and \ref{eq:T} with the exception of the first entry which is $TR_{00} = 0$, $TC_{00} = 0$ and $T_{00} =$ 'done'. The following animation show us the initialisiaton of all entries of our 4 matrices. In black we have the values of the score matrix ($H_{ij}$) and traceback matrix ($T_{ij}$), in <font color='blue'>blue</font> and in <font color='green'>green</font>  it is represented the entries of matrix $TR$ and $TC$ respectively.


<img id="img3" src="tnw_initialisation_gif.gif">

<a onClick="resetGif('img3')">reset gif initialisation</a>

<script type="text/javascript">

// reset an animated gif to start at first image without reloading it from server.
// Note: if you have the same image on the page more than ones, they all reset.
function resetGif(id) {
    var img = document.getElementById(id);
    var imageUrl = img.src;
    img.src = "";
    img.src = imageUrl;
};

</script>


#### 2. Calculations

<img id="img3" src="tnw_calculations_gif.gif">

<a onClick="resetGif('img3')">reset gif calculations</a>

<script type="text/javascript">

// reset an animated gif to start at first image without reloading it from server.
// Note: if you have the same image on the page more than ones, they all reset.
function resetGif(id) {
    var img = document.getElementById(id);
    var imageUrl = img.src;
    img.src = "";
    img.src = imageUrl;
};

</script>



In [1]:
a = 2
b = 2
c = a+b

In [2]:
print(c)

4
